**Libreria <a href="https://pandas.pydata.org/docs/getting_started/index.html"> Pandas**</a>

*Permite el procesamiento de datos sobre dataframes*

In [1]:
import pandas as pd

**Libreria  <a href="https://docs.python.org/3.12/library/ast.html"> Abstract Syntax Trees**</a>

*Permite la lectura literal de la linea*

In [2]:
import ast

# **Lectura**  

In [3]:
list = []
for line in open('G:\\My Drive\\IT\\dataset\\australian_user_reviews.json', encoding='utf-8').readlines(): # australian_user_reviews.json
    list.append(ast.literal_eval(line))
df_australian_User_Reviews = pd.DataFrame.from_dict(list)

In [4]:
list = []
for line in open('G:\\My Drive\\IT\\dataset\\australian_users_items.json', encoding='utf-8').readlines(): # australian_users_items.json
    list.append(ast.literal_eval(line))
df_items = pd.DataFrame.from_dict(list)

In [5]:
df_steam = pd.read_json('G:\\My Drive\\IT\\dataset\\output_steam_games.json', lines=True) #output_steam_games.json

# **Transform**

**`df_User_Reviews`**

In [4]:
#Creo df_User_Reviews'
df_Reviews = df_australian_User_Reviews[['user_id','reviews']]

In [5]:
#Desanido los valores de 'reviews'
df_Reviews = df_Reviews.explode('reviews')

In [6]:
#Reseteo el index 
df_Reviews.reset_index(drop=True, inplace=True)

In [7]:
#Normalizo la columna 'reviews'
reviews_normalizado = pd.json_normalize(df_Reviews['reviews'])

In [8]:
#Concateno la columna 'reviews' normalizada, con el df original.
df_Reviews = pd.concat([df_Reviews.drop(columns=['reviews']), reviews_normalizado], axis=1)

In [9]:
#Elimino columnas 'last_edited' y 'funny'
df_Reviews = df_Reviews[['user_id','posted','item_id','helpful','recommend','review']]

In [24]:
df_Reviews['posted'].to_frame().replace('.','',inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12668\805601059.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_Reviews['posted'].to_frame().replace('.','',inplace=True)


In [36]:
df_Reviews

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


In [27]:
df_Reviews['posted'].replace(".",'',inplace=True)

In [42]:
#Modifico el valor de la clave 'posted'    
df_Reviews['posted'] = df_Reviews['posted'].replace("Posted ","") #Reemplazo 'Posted' --> ''
#df_Reviews['posted'] = df_Reviews['posted']. #Reemplazo '.' --> ''

In [38]:
df_Reviews

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...
59328,76561198312638244,Posted July 10.,70,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,Posted July 8.,362890,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,Posted July 3.,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,Posted July 20.,730,No ratings yet,True,:D


*Importo `datetime`, para cambiar el formato de los valores de 'Posted' a '%d-%m-%Y'* 

In [10]:
from datetime import datetime

In [11]:
#Convierto a datatime el tipo de la clave 'posted'
def format(dic):
    try:
        #Try to convert
        dic['posted']=(datetime.strptime(dic['posted'],'%B %d, %Y')).strftime('%d-%m-%Y')
        return dic
    except ValueError:
        #catch error
        return None
    
df_User_Reviews['reviews']=df_User_Reviews['reviews'].apply(lambda row: format(row))

In [12]:
#Elimino NaNs 
df_User_Reviews.dropna(inplace=True)

In [62]:
df_User_Reviews

,user_id,reviews
0,76561197970982479,"{'posted': '05-11-2011', 'item_id': '1250', 'h..."
1,76561197970982479,"{'posted': '15-07-2011', 'item_id': '22200', '..."
2,76561197970982479,"{'posted': '21-04-2011', 'item_id': '43110', '..."
3,js41637,"{'posted': '24-06-2014', 'item_id': '251610', ..."
4,js41637,"{'posted': '08-09-2013', 'item_id': '227300', ..."
...,...,...
59280,wayfeng,"{'posted': '14-10-2015', 'item_id': '730', 'he..."
59283,76561198251004808,"{'posted': '10-10-2015', 'item_id': '253980', ..."
59293,72947282842,"{'posted': '31-10-2015', 'item_id': '730', 'he..."
59295,ApxLGhost,"{'posted': '14-12-2015', 'item_id': '730', 'he..."


In [16]:
#Convierto a entero el tipo de dato de la clave 'item_id'
def to_Int(dic):
    try:
        #Try to convert
        dic['item_id'] = int(dic['item_id']) 
        return dic
    except ValueError:
        #catch error
        return None
    
df_User_Reviews['reviews'] = (df_User_Reviews['reviews'].apply(lambda row: to_Int(row))).dropna()

In [17]:
df_User_Reviews['reviews']

0        {'posted': '05-11-2011', 'item_id': 1250, 'hel...
1        {'posted': '15-07-2011', 'item_id': 22200, 'he...
2        {'posted': '21-04-2011', 'item_id': 43110, 'he...
3        {'posted': '24-06-2014', 'item_id': 251610, 'h...
4        {'posted': '08-09-2013', 'item_id': 227300, 'h...
                               ...                        
59280    {'posted': '14-10-2015', 'item_id': 730, 'help...
59283    {'posted': '10-10-2015', 'item_id': 253980, 'h...
59293    {'posted': '31-10-2015', 'item_id': 730, 'help...
59295    {'posted': '14-12-2015', 'item_id': 730, 'help...
59304    {'posted': '29-12-2015', 'item_id': 369200, 'h...
Name: reviews, Length: 49186, dtype: object

In [18]:
#Modifico tipo de dato boolean --> int sobre la columna 'recommend' 
def to_Int_Boolean(dic):
    try:
        #Intento
        if (dic['recommend']):
            #recommend == True
            dic['recommend'] = 1
        else:
            #Recommen == False
            dic['recommend'] = 0
    except ValueError:
        #Capturo el error
        return None

df_User_Reviews['reviews'].apply(lambda row: to_Int_Boolean(row))

0        None
1        None
2        None
3        None
4        None
         ... 
59280    None
59283    None
59293    None
59295    None
59304    None
Name: reviews, Length: 49186, dtype: object

In [19]:
#Elimino NaNs 
df_User_Reviews.dropna(inplace=True)

**Feature Engineering: Sentiments Analysis con <a href="https://www.nltk.org/"> NLTK </a>** 
 

*Descargo<a href="https://www.nltk.org/nltk_data/"> `VADER Sentiment Lexicon` </a>* 

*Contiene un conjunto de palabras con sus respectivos puntajes de polaridad, lo que ayuda a determinar la intensidad del sentimiento expresado en una frase.*

In [20]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [21]:
# Importo clase SentimentIntensityAnalyzer   
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

In [22]:
# Instancio SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer() 

In [23]:
# Implemento funcion 'polarity_scores' sobre df_User_Reviews
df_User_Reviews['sentiment_analysis'] = df_User_Reviews['reviews'].apply(lambda x: sia.polarity_scores(x['review'])['compound']) #Espicifico la clave 'compound', porque resume la intesidad del sentimiento expresado en la frase 

In [24]:
# Recategorizo la intensidad del sentimiento expresado en cada frase
""" Nueva Escala
        
        POSITIVO -> 2
        NEUTRO -> 1
        NEGATIVO -> 0

 """ 

def recategorizar(int):
    if(int>0):
        return 2
    else:
        if(abs(int) == 0):
            return 1
        else:
            return 0 
        
df_User_Reviews['sentiment_analysis'] = df_User_Reviews['sentiment_analysis'].apply(lambda x: recategorizar(x))

**`df_Users_Items`**

In [25]:
#Creo df_Users_Items'
df_Users_Items = df_items[['user_id','items_count','items']]

In [26]:
#Desanido 'items'
df_Users_Items=df_Users_Items.explode('items')

In [27]:
#Reseteo Index
df_Users_Items.reset_index(drop=True, inplace=True)

In [28]:
#Cantidad de filas con datos nulos en la columna 'items'
df_Users_Items['items'].isna().sum()

16806

In [29]:
#Elimino nulos
df_Users_Items.dropna(inplace=True) 

In [30]:
#Elimino clave 'playtime_2weeks', porque no es un dato relevante en el desarrollo del trabajo y reduce el tamaño del df. 
df_Users_Items['items'].apply(lambda x : x.pop('playtime_2weeks'))

0          0
1          0
2          0
3          0
4          0
          ..
5170009    0
5170010    0
5170011    3
5170012    4
5170013    3
Name: items, Length: 5153209, dtype: int64

In [31]:
#Modifico tipo de dato, str --> int
#Motivo, conociendo los tipos de valores de clave es mas sencillo trabajar con int, antes que str.
def toInt(dic):
    dic['item_id']=int(dic['item_id'])

df_Users_Items['items'].apply(lambda x: toInt(x))

0          None
1          None
2          None
3          None
4          None
           ... 
5170009    None
5170010    None
5170011    None
5170012    None
5170013    None
Name: items, Length: 5153209, dtype: object

In [32]:
def itemsId_Null(fila):
    return fila['items']['item_name']==''

df_Users_Items.apply(itemsId_Null, axis=1).sum()

0

**`df_Steam_Games`**

In [33]:
#Creo df unicamente con los siguientes campos: {genres, app_name, id}
#Porque el resto de la columna no aportan datos relevantes para la implementacion de las funciones con la API.
df_Steam_Games = df_steam[['genres','app_name','id']]

In [34]:
#Consulto cantidad de nulos en la columna 'genres'
df_Steam_Games['genres'].isna().sum()

91593

In [35]:
#Consulto cantidad de nulos en la columna 'app_name'
df_Steam_Games['app_name'].isna().sum()

88312

In [36]:
#Consulto cantidad de nulos en la columna 'id'
df_Steam_Games['id'].isna().sum()

88312

In [37]:
#Elimino nulos 
df_Steam_Games.dropna(inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7352\1082775196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Steam_Games.dropna(inplace=True)


In [38]:
#Reset index
df_Steam_Games.reset_index(drop=True, inplace=True)

In [39]:
df_Steam_Games

,genres,app_name,id
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,761140.0
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,643980.0
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,670290.0
3,"[Action, Adventure, Casual]",弹炸人2222,767400.0
4,"[Action, Adventure, Simulation]",Battle Royale Trainer,772540.0
...,...,...,...
28845,"[Action, Adventure, Casual, Indie]",Kebab it Up!,745400.0
28846,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,773640.0
28847,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,733530.0
28848,"[Indie, Racing, Simulation]",Russian Roads,610660.0


In [40]:
#Convertir a entero la comlumna 'id'
def to_Int(nro):
    try:
        #Try to convert
        return int(nro)
    except ValueError:
        #catch error
        return None
    
df_Steam_Games['id'] = df_Steam_Games['id'].apply(lambda x: to_Int(x)).dropna()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7352\1772122932.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Steam_Games['id'] = df_Steam_Games['id'].apply(lambda x: to_Int(x)).dropna()


# **Load**

*Cargo los dataframes procesados en formato .parquet por el tamaño del archivo resultante que devuelve este tipo de compresion.*

**`df_User_Reviews`**

In [42]:
df_User_Reviews.to_parquet('user_Reviews')

**`df_Users_Items`**

In [43]:
df_Users_Items.to_parquet('user_items')

**`df_Steam_Games`**

In [44]:
df_Steam_Games.to_parquet('steam_Games')

In [51]:
df_Users_Items['items'][0].keys()

dict_keys(['item_id', 'item_name', 'playtime_forever'])

In [54]:
pd.merge(df_Steam_Games, df_Users_Items['items'], left_on='id', right_on='item_id')

KeyError: 'item_id'